## Audio Enabled GenAI Application

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
import base64
import requests
load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

### Setting up the AI Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential()
)

### Setting up the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client(api_version="2024-10-21")

### Encoding the Audio Input

In [ ]:
# Encode the audio file
file_path = "https://raw.githubusercontent.com/kuljotSB/AI-102-Certification/raw/refs/heads/main/AzureLanguageService/Audio-Enabled-GenAI-App/avocados.mp3"
response = requests.get(file_path)
response.raise_for_status()
audio_data = base64.b64encode(response.content).decode('utf-8')

### Sending a Request to the Model Deployment

In [ ]:
message = openai_client.messages.create(
    model = model_deployment_name,
    messages=[
       {"role": "system", "content": "You are a helpful Audio-Enabled AI assistant."},
       { "role": "user",
           "content": [
           { 
               "type": "text",
               "text": "Summarize the audio content."
           },
           {
               "type": "input_audio",
               "input_audio": {
                   "data": audio_data,
                   "format": "mp3"
               }
           }
       ] }
   ],
    max_tokens=1024
)

json_message = message.to_dict()
print(json_message['content'][0]['text'])